In [1]:
%matplotlib inline
from __future__ import division

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from operator import truediv
from nxpd import draw
from collections import Counter
import itertools

In [2]:
BREAKOUT_REQ_RATIO = 2
BREAKOUT_REQ_FRACTION = 0.2

G = nx.DiGraph()

In [3]:
def insert_node(name, email, learn, teach):
    G.add_node(name, email=email, learn=learn, teach=teach)
    
    for node in G.nodes(data=True):
        learn_other = node[1]['learn']
        teach_other = node[1]['teach']
        
        common = [l for l in learn if l in teach_other]
        w = len(common)
        G.add_edge(name, node[0], weight=w, common=common)
        
        common = [l for l in teach if l in learn_other]
        w = len(common)
        G.add_edge(node[0], name, weight=w, common=common)

In [4]:
# Retrieve data from Google Sheet and parse using pandas dataframe
data = pd.read_csv("https://raw.githubusercontent.com/benelson/collaboratr/master/collaboratr.csv")
data = data.replace(np.nan,' ', regex=True)
users = data['1. What is your name?']
emails = data['2. What is your email?']
teach = data['3. What can you help others with?']
learn = data['4. What do you want to collaborate on or learn about?']

## Decision Tree

In [5]:
def find_breakouts(teach, learn):
    """
    Find subjects which deserve breakouts depending 
    on student to teach ratios.
    """
    big_list_teach = []
    for i in range(len(teach)):
        big_list_teach.append(teach[i].split(';'))

    big_list_learn = []
    for i in range(len(learn)):
        big_list_learn.append(learn[i].split(';'))
        
    teach_dict = dict(Counter(list((itertools.chain.from_iterable(big_list_teach)))))
    learn_dict = dict(Counter(list((itertools.chain.from_iterable(big_list_learn)))))
    
    common_names = [ n for n in set(teach_dict).intersection(set(learn_dict)) ]

    teach_dict = {k:teach_dict[k] for k in common_names if k in teach_dict} # Python3
    learn_dict = {k:learn_dict[k] for k in common_names if k in learn_dict} # Python3
    
    N_teach = list(teach_dict.values())
    N_learn = list(learn_dict.values())
    
    ratios =  list(map(truediv,N_learn,N_teach))

    N = len(users)
    subjects = list(teach_dict.keys())
    isBreakout = (np.array(ratios) > BREAKOUT_REQ_RATIO) & (np.array(N_learn) > BREAKOUT_REQ_FRACTION*N)
    breakouts = np.array(subjects)[list(np.where(isBreakout)[0])]
    
    return breakouts

In [6]:
breakouts = find_breakouts(teach, learn)
print(breakouts)

['GAIA DRI' 'gaussian processes' 'packaging (Python)' 'autodiff']


## Graphs

In [19]:
for u,e,l,t in zip(users, emails, learn, teach):
    insert_node(u, email=e, learn=l.split(';'), teach=t.split(';'))

In [20]:
participants = G.nodes(data=True)

In [11]:
def assign_users():
    """
    This uses LOTS of nested loops. Work to be done here.
    """
    assign = {}
    i=0
    for p in participants:
        assign_p = {}
        for l in p[1]['learn']:
            if l in breakouts:
                assign_p[l] = 'breakout'
            else:
                edges = G.edges(p[0], data=True)
                for e in edges:
                    if (l in e[2]['common']) and (l!=''):
                        assign_p[l] = e[1]
                    else: 
                        pass
        assign[p[0]] = assign_p
        i+=1
        
    return assign

In [12]:
assign = assign_users()

In [13]:
G_new = nx.DiGraph(bgcolor='white')

In [14]:
G_new.add_nodes_from(participants, style='filled', fillcolor='black', 
                     fontcolor='white', fontname='sans-serif', fontsize=24)
G_new.add_node('breakout', shape='rectangle', width=4, height=1, style='filled', fillcolor='black',
              fontcolor='white', fontname='sans-serif', fontsize=30)

In [15]:
for p in G.nodes():
    mappings = assign[p]
    for m in mappings.keys():
        G_new.add_edge(p, mappings[m], label=m, style='dotted', fontcolor='red', fontsize=20,
                      fontname='sans-serif')

In [16]:
draw(G_new, filename='0.svg')

InvocationException: GraphViz's executables not found

# Customized Graphs

In [ ]:
participants = G_new.nodes()
connections = G_new.edges(data=True)

In [ ]:
index=1
for p in participants:
    con = [c for c in connections if c[0] == p]
    links = [c[1] for c in con]
    links.append(p)

    H = nx.DiGraph()
    H.add_nodes_from(links, style='filled', fillcolor='black', 
                         fontcolor='white', fontname='sans-serif', fontsize=24)

    for c in con:
        H.add_edge(c[0], c[1], label=c[2]['label'], style='dotted', fontcolor='red', fontsize=12,
                          fontname='sans-serif')te

    draw(H, filename='Resources/'+str(index).zfill(2)+'.svg')
    index+=1

# Creating a Web Page

In [ ]:
from jinja2 import Template

In [ ]:
# Define jinja2 template
TEMPLATE = Template("""<html><body></body><html>{% for i in range(0,index)%}<center><img src="Resources/{{i}}.svg" align="middle"></center><br><br><br><br><br><br>{% endfor %}""")

In [ ]:
with open('display.html', 'w') as file:
    file.write(TEMPLATE.render(index=index))